Imports

In [1]:
!pip install -q scikit-learn pandas tqdm kaggle matplotlib

In [2]:
!sudo apt -q install zip unzip

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 16 not upgraded.
Need to get 335 kB of archives.
After this operation, 1231 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 unzip amd64 6.0-25ubuntu1.1 [168 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 zip amd64 3.0-11build1 [167 kB]
Fetched 335 kB in 0s (1155 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 17208 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-25ubuntu1.1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######....................................................] 8Unpacking unzip (6.0-25ubuntu1.1) ...
7Progress: [

In [3]:
!chmod 600 kaggle.json  # Ensure the file is private
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
# # Download the dataset
# !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

# # Extract the dataset
# !tar -xzvf food-101.tar.gz

In [5]:
!kaggle datasets download -d dansbecker/food-101

100%|██████████████████████████████████████| 9.38G/9.38G [05:52<00:00, 34.2MB/s]
100%|██████████████████████████████████████| 9.38G/9.38G [05:52<00:00, 28.5MB/s]


In [6]:
!unzip -q food-101.zip -d food-101

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torchvision import datasets, transforms, models
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle
from datetime import datetime

In [8]:
model_save_dir = 'Trained_Models'
os.makedirs(model_save_dir, exist_ok=True)  # Create the directory if it doesn't exist

Data Loading and Preprocessing

In [9]:
import os
import shutil
import numpy as np

# Define the paths and split ratios
source_dir = 'food-101/food-101/food-101/images'  # Replace with source directory path
destination_dir = 'data'  # Replace with destination directory path
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Create output directory
os.makedirs(destination_dir, exist_ok=True)

# Create train, val, and test directories in the destination folder
for split in ['train', 'val', 'test']:
    split_dir = os.path.join(destination_dir, split)
    os.makedirs(split_dir, exist_ok=True)

# Move files into splits
for class_dir in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_dir)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        np.random.shuffle(images)
        
        train_split = int(train_ratio * len(images))
        val_split = int(val_ratio * len(images))

        # Creating subdirectories for each class in train, val, and test in the destination folder
        for split in ['train', 'val', 'test']:
            split_class_dir = os.path.join(destination_dir, split, class_dir)
            os.makedirs(split_class_dir, exist_ok=True)
        
        # Copy images to respective directories in the destination folder
        for i, image in enumerate(images):
            if i < train_split:
                split = 'train'
            elif i < train_split + val_split:
                split = 'val'
            else:
                split = 'test'

            src_path = os.path.join(class_path, image)
            dest_path = os.path.join(destination_dir, split, class_dir, image)
            shutil.copy(src_path, dest_path)


In [10]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define your transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Create datasets
train_dataset = datasets.ImageFolder(root=os.path.join(destination_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(destination_dir, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(destination_dir, 'test'), transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=8)

Model Definitions

In [13]:
def get_model(pretrained=True):
    return {
        'EfficientNetV2': models.efficientnet_v2_s(pretrained=pretrained),
        'ResNet50': models.resnet50(pretrained=pretrained),
        'VGG16': models.vgg16(pretrained=pretrained),
        'MobileNetV3': models.mobilenet_v3_large(pretrained=pretrained),
        'DenseNet121': models.densenet121(pretrained=pretrained)
    }

def modify_model_classes(model_dict, num_classes):
    for name, model in model_dict.items():
        if name == 'EfficientNetV2':
            num_ftrs = model.classifier[1].in_features
            model.classifier[1] = torch.nn.Linear(num_ftrs, num_classes)
        elif name == 'ResNet50':
            num_ftrs = model.fc.in_features
            model.fc = torch.nn.Linear(num_ftrs, num_classes)
        elif name == 'VGG16':
            num_ftrs = model.classifier[6].in_features
            model.classifier[6] = torch.nn.Linear(num_ftrs, num_classes)
        elif name == 'MobileNetV3':
            num_ftrs = model.classifier[3].in_features
            model.classifier[3] = torch.nn.Linear(num_ftrs, num_classes)
        elif name == 'DenseNet121':
            num_ftrs = model.classifier.in_features
            model.classifier = torch.nn.Linear(num_ftrs, num_classes)
    return model_dict

In [14]:
pretrained_models = modify_model_classes(get_model(pretrained=True), num_classes=101)
non_pretrained_models = modify_model_classes(get_model(pretrained=False), num_classes=101)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 104MB/s] 
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are depre

Training and Evaluation

In [15]:
def train_and_validate_model(model, train_loader, val_loader, epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if torch.cuda.device_count() > 1:
      print("Using", torch.cuda.device_count(), "GPUs!")
      model = nn.DataParallel(model)

    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    scaler = GradScaler()

    best_val_accuracy = 0

    # Lists to store per epoch metrics
    epoch_train_losses = []
    epoch_train_accuracies = []
    epoch_val_losses = []
    epoch_val_accuracies = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_accuracy = train_correct / train_total
        epoch_train_losses.append(train_loss / len(train_loader))
        epoch_train_accuracies.append(train_accuracy)

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = val_correct / val_total

        epoch_val_losses.append(val_loss / len(val_loader))
        epoch_val_accuracies.append(val_accuracy)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {val_accuracy}')

        # Update best model if validation accuracy improves
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_wts = model.state_dict()

        scheduler.step()

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_train_losses, epoch_train_accuracies, epoch_val_losses, epoch_val_accuracies


In [16]:
def evaluate_model(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    accuracy = accuracy_score(all_labels, all_preds)

    return precision, recall, f1, accuracy

In [17]:
def train_and_evaluate_models(model_dict, pretrained_status, epochs=10):
    results_df = pd.DataFrame(columns=['Model', 'Pretrained', 'Precision', 'Recall', 'F1 Score', 'Accuracy'])

    metrics_dict = {}
    
    for name, model in model_dict.items():
        print(f"Training and validating {name} (Pretrained: {pretrained_status})")
        trained_model, train_losses, train_accuracies, val_losses, val_accuracies = train_and_validate_model(model, train_loader, val_loader, epochs=epochs)

        # Store metrics for plotting
        metrics_dict[name] = {
            'train_losses': train_losses,
            'train_accuracies': train_accuracies,
            'val_losses': val_losses,
            'val_accuracies': val_accuracies
        }

        # Saving the trained model
        model_path = os.path.join(model_save_dir, f'{name}_{"pretrained" if pretrained_status else "non_pretrained"}.pt')
        torch.save(trained_model.state_dict(), model_path)
        print(f"Saved trained model at {model_path}")
        
        print(f"Evaluating {name} (Pretrained: {pretrained_status})")
        precision, recall, f1, accuracy = evaluate_model(trained_model, test_loader)
        print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}, Accuracy: {accuracy}")
        results_df.loc[len(results_df.index)] = [name, pretrained_status, precision, recall, f1, accuracy]
        
    return results_df, metrics_dict

In [ ]:
# Train and evaluate pretrained models
pretrained_results_df, pretrained_metrics = train_and_evaluate_models(pretrained_models, True)

In [21]:
pretrained_results_df

,Model,Pretrained,Precision,Recall,F1 Score,Accuracy
0,EfficientNet,True,0.829306,0.827624,0.827476,0.827624
1,ResNet,True,0.751379,0.748020,0.748540,0.748020
2,VGG,True,0.317313,0.330099,0.319110,0.330099
3,MobileNet,True,0.779271,0.777822,0.777975,0.777822
4,DenseNet,True,0.799960,0.798317,0.798242,0.798317


In [22]:
current_timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
filename = f'results_pretrained_{current_timestamp}.csv'
pretrained_results_df.to_csv(filename)

In [18]:
# Train and evaluate non-pretrained models
non_pretrained_results_df, non_pretrained_metrics = train_and_evaluate_models(non_pretrained_models, False)

Training and validating EfficientNet (Pretrained: False)


100%|██████████| 277/277 [01:18<00:00,  3.51it/s]


Epoch 1/10, Train Loss: 4.375394216943734, Val Loss: 4.228005222127408, Val Accuracy: 0.05638613861386139


100%|██████████| 277/277 [01:17<00:00,  3.59it/s]


Epoch 2/10, Train Loss: 3.8542740870038523, Val Loss: 3.7903696464586862, Val Accuracy: 0.11861386138613861


...

Epoch 8/10, Train Loss: 1.0241381676618804, Val Loss: 1.4399222588237328, Val Accuracy: 0.6272772277227723


100%|██████████| 277/277 [01:15<00:00,  3.69it/s]


Epoch 9/10, Train Loss: 0.8998948043003839, Val Loss: 1.4025480694408659, Val Accuracy: 0.6384653465346535


100%|██████████| 277/277 [01:15<00:00,  3.69it/s]


Epoch 10/10, Train Loss: 0.8340397582587783, Val Loss: 1.3974817772454853, Val Accuracy: 0.6416831683168317
Saved trained model at /Trained_Models/DenseNet_non_pretrained.pt
Evaluating DenseNet (Pretrained: False)
Precision: 0.6457570051936392, Recall: 0.6420792079207921, F1 Score: 0.6412335839847823, Accuracy: 0.6420792079207921


In [19]:
non_pretrained_results_df

,Model,Pretrained,Precision,Recall,F1 Score,Accuracy
0,EfficientNet,False,0.574821,0.574257,0.571603,0.574257
1,ResNet,False,0.563733,0.552376,0.551026,0.552376
2,VGG,False,0.000098,0.009901,0.000194,0.009901
3,MobileNet,False,0.521485,0.520198,0.517134,0.520198
4,DenseNet,False,0.645757,0.642079,0.641234,0.642079


In [20]:
current_timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
filename = f'results_nonpretrained_{current_timestamp}.csv'
non_pretrained_results_df.to_csv(filename)

In [23]:
with open(f'pt_metrics_{current_timestamp}.pickle', 'wb') as handle:
    pickle.dump(pretrained_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open(f'npt_metrics_{current_timestamp}.pickle', 'wb') as handle:
    pickle.dump(non_pretrained_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Plotting
for model_name in pretrained_models.keys():
    pt_epochs_range = range(1, 10 + 1)
    npt_epochs_range = range(1, 20 + 1)

    plt.figure(figsize=(12, 6))

    # Plot for pretrained model
    plt.subplot(1, 2, 1)
    plt.plot(pt_epochs_range, pretrained_metrics[model_name]["train_losses"], color='blue', label='Training Loss')
    plt.plot(pt_epochs_range, pretrained_metrics[model_name]["val_losses"], color='red', label='Validation Loss')
    plt.plot(pt_epochs_range, pretrained_metrics[model_name]["train_accuracies"], color='orange', label='Training Accuracy')
    plt.plot(pt_epochs_range, pretrained_metrics[model_name]["val_accuracies"], color='green', label='Validation Accuracy')
    plt.title(f'{model_name} Pretrained')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()

    # Plot for non-pretrained model
    plt.subplot(1, 2, 2)
    plt.plot(npt_epochs_range, non_pretrained_metrics[model_name]["train_losses"], color='blue', label='Training Loss')
    plt.plot(npt_epochs_range, non_pretrained_metrics[model_name]["val_losses"], color='red', label='Validation Loss')
    plt.plot(npt_epochs_range, non_pretrained_metrics[model_name]["train_accuracies"], color='orange', label='Training Accuracy')
    plt.plot(npt_epochs_range, non_pretrained_metrics[model_name]["val_accuracies"], color='green', label='Validation Accuracy')
    plt.title(f'{model_name} Non-Pretrained')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()

    plt.tight_layout()
    plt.show()